In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim

## **Load and Prepare Data**

In [2]:
import import_ipynb
import dataset
from dataset import parse_dataset

importing Jupyter notebook from dataset.ipynb


In [3]:
ids_dataset = dataset.DatasetIDS2018()

<string>:3: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.


In [4]:
columns_to_drop = ['Timestamp', 'Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port']

In [5]:
labels = [
    'Benign',
    'Bot',
    'Brute Force -XSS',
    'DDOS attack-HOIC',
    'DDOS attack-LOIC-UDP',
    'DDoS attacks-LOIC-HTTP',
    'DoS attacks-GoldenEye',
    'DoS attacks-Hulk',
    'DoS attacks-Slowloris',
    'FTP-BruteForce',
    'SSH-Bruteforce',
    'Label',
    'Brute Force -Web',
    'DoS attacks-SlowHTTPTest',
    'Infilteration',
    'SQL Injection'                      
]

labels_emb = {}
for i, label in enumerate(labels):
    labels_emb[label] = i

Train data will consists of classes:
- Benign                      
- Bot                                           
- Brute Force -XSS                 
- DDOS attack-HOIC              
- DDOS attack-LOIC-UDP            
- DDoS attacks-LOIC-HTTP      
- DoS attacks-GoldenEye          
- DoS attacks-Hulk                    
- DoS attacks-Slowloris          
- FTP-BruteForce                                
- Label                                                
- SSH-Bruteforce 


In [6]:
train_labels = [
    'Benign',
    'Bot',
    'Brute Force -XSS',
    'DDOS attack-HOIC',
    'DDOS attack-LOIC-UDP',
    'DDoS attacks-LOIC-HTTP',
    'DoS attacks-GoldenEye',
    'DoS attacks-Hulk',
    'DoS attacks-Slowloris',
    'FTP-BruteForce',
    'SSH-Bruteforce',
    'Label',
                          
]
train = ids_dataset.get_data_by_labels(labels=train_labels)
train


Index(['Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Tot Fwd Pkts',
       'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
       'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags',
       'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s',
       'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean',
       'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt',
       'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt',
       'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg',
      

In [7]:
train_dataset = parse_dataset(dataset=train, columns_to_drop=columns_to_drop, fixed_type=float, labels_emb=labels_emb, labels_column_name='Label')
train_dataset

,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,6.0,141385.0,9.0,7.0,553.0,3773.0,202.0,0.0,61.444444,87.534438,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,6.0,281.0,2.0,1.0,38.0,0.0,38.0,0.0,19.000000,26.870058,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,6.0,279824.0,11.0,15.0,1086.0,10527.0,385.0,0.0,98.727273,129.392497,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,6.0,132.0,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,6.0,274016.0,9.0,13.0,1285.0,6141.0,517.0,0.0,142.777778,183.887722,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8284223,6.0,104258.0,2.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
8284224,6.0,103022.0,2.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
8284229,6.0,105445.0,2.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
8284232,6.0,733880.0,2.0,2.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


Test data will consists of classes:
- Brute Force -Web                 
- DoS attacks-SlowHTTPTest      
- Infilteration                                              
- SQL Injection                     


In [8]:
test_labels = [label for label in labels if label not in train_labels]
test = ids_dataset.get_data_by_labels(labels=test_labels)
test

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
1048672,21,6,16/02/2018 10:12:14,21,1,1,0,0,0,0,...,40,0,0.0,0,0,0.0,0.0,0,0,DoS attacks-SlowHTTPTest
1048673,21,6,16/02/2018 10:12:14,3,1,1,0,0,0,0,...,40,0,0.0,0,0,0.0,0.0,0,0,DoS attacks-SlowHTTPTest
1048674,21,6,16/02/2018 10:12:14,3,1,1,0,0,0,0,...,40,0,0.0,0,0,0.0,0.0,0,0,DoS attacks-SlowHTTPTest
1048675,21,6,16/02/2018 10:12:14,3,1,1,0,0,0,0,...,40,0,0.0,0,0,0.0,0.0,0,0,DoS attacks-SlowHTTPTest
1048676,21,6,16/02/2018 10:12:14,2,1,1,0,0,0,0,...,40,0,0.0,0,0,0.0,0.0,0,0,DoS attacks-SlowHTTPTest
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8284248,20000,6,28/02/2018 10:50:04,2,1,1,0,0,0,0,...,24,0,0,0,0,0,0.0,0,0,Infilteration
8284249,23,6,28/02/2018 11:59:12,3,1,1,0,0,0,0,...,24,0,0,0,0,0,0.0,0,0,Infilteration
8284250,425,6,28/02/2018 10:50:04,2,1,1,0,0,0,0,...,24,0,0,0,0,0,0.0,0,0,Infilteration
8284252,23,6,28/02/2018 11:10:50,22,1,1,0,0,0,0,...,24,0,0,0,0,0,0.0,0,0,Infilteration


In [9]:
test_dataset = parse_dataset(dataset=test, columns_to_drop=columns_to_drop, fixed_type=float, labels_emb=labels_emb, labels_column_name='Label')
test_dataset

,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
1048672,6,21,1,1,0,0,0,0,0.0,0.0,...,40,0.0,0.0,0,0,0.0,0.0,0,0,13
1048673,6,3,1,1,0,0,0,0,0.0,0.0,...,40,0.0,0.0,0,0,0.0,0.0,0,0,13
1048674,6,3,1,1,0,0,0,0,0.0,0.0,...,40,0.0,0.0,0,0,0.0,0.0,0,0,13
1048675,6,3,1,1,0,0,0,0,0.0,0.0,...,40,0.0,0.0,0,0,0.0,0.0,0,0,13
1048676,6,2,1,1,0,0,0,0,0.0,0.0,...,40,0.0,0.0,0,0,0.0,0.0,0,0,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8284248,6,2,1,1,0,0,0,0,0.0,0.0,...,24,0.0,0.0,0,0,0.0,0.0,0,0,14
8284249,6,3,1,1,0,0,0,0,0.0,0.0,...,24,0.0,0.0,0,0,0.0,0.0,0,0,14
8284250,6,2,1,1,0,0,0,0,0.0,0.0,...,24,0.0,0.0,0,0,0.0,0.0,0,0,14
8284252,6,22,1,1,0,0,0,0,0.0,0.0,...,24,0.0,0.0,0,0,0.0,0.0,0,0,14


In [10]:
test_dataset.to_pickle('../data/pickle/test_dataset.pkl')
train_dataset.to_pickle('../data/pickle/train_dataset.pkl')

In [13]:
test_dataset = data.TensorDataset(torch.from_numpy(test_dataset.values).float(),torch.from_numpy(test_dataset.values[:,-1].astype(float)).float())
train_dataset = data.TensorDataset(torch.from_numpy(train_dataset.values).float(),torch.from_numpy(train_dataset.values[:,-1].astype(float)).float())

In [14]:
train_data_loader = data.DataLoader(train_dataset, batch_size=256, shuffle=True) 
test_data_loader = data.DataLoader(test_dataset, batch_size=256, shuffle=True) 

In [15]:
next(iter(train_data_loader))

[tensor([[6.0000e+00, 4.2344e+06, 5.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [6.0000e+00, 1.0800e+02, 2.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [6.0000e+00, 8.6122e+07, 2.0000e+00,  ..., 8.6122e+07, 8.6122e+07,
          0.0000e+00],
         ...,
         [6.0000e+00, 5.5943e+05, 6.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [6.0000e+00, 1.4100e+02, 2.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [6.0000e+00, 3.5000e+01, 1.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00]]),
 tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  7.,  0.,  0.,  0.,  3.,  0.,
          9.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  7.,  0., 10.,  0.,  0.,  0.,
          0.,  0.,  3.,  3.,  0.,  0.,  0.,  3.,  0.,  0.,  3.,  0.,  0.,  0.,
          0.,  7.,  0.,  0.,  0.,  3.,  0.,  3.,  7.,  0.,  0.,  0.,  3.,  1.,
          0.,  9.,  0.,  0.,  0.,  0.,  0.,  0.,  3.,  0.,  1.,  9.,  0., 

In [16]:
next(iter(test_data_loader))

[tensor([[  6.,   2.,   1.,  ...,   0.,   0.,  14.],
         [ 17., 508.,   1.,  ...,   0.,   0.,  14.],
         [  6.,  27.,   1.,  ...,   0.,   0.,  14.],
         ...,
         [  6.,   3.,   1.,  ...,   0.,   0.,  13.],
         [  6.,  45.,   1.,  ...,   0.,   0.,  14.],
         [  6.,   2.,   1.,  ...,   0.,   0.,  13.]]),
 tensor([14., 14., 14., 13., 14., 14., 14., 13., 14., 14., 14., 13., 14., 14.,
         13., 12., 13., 14., 14., 14., 14., 13., 13., 13., 14., 13., 14., 14.,
         13., 13., 14., 14., 14., 14., 13., 14., 14., 14., 13., 13., 13., 14.,
         13., 13., 14., 14., 13., 14., 13., 14., 13., 13., 14., 14., 13., 13.,
         14., 14., 14., 14., 13., 13., 13., 13., 13., 13., 13., 13., 14., 14.,
         14., 14., 13., 14., 13., 13., 14., 14., 14., 14., 13., 14., 13., 14.,
         14., 14., 14., 13., 13., 14., 13., 14., 14., 14., 14., 14., 14., 13.,
         14., 13., 13., 14., 14., 14., 13., 14., 14., 14., 13., 14., 13., 13.,
         13., 13., 13., 13., 13., 

## **Neural Network**

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, input_dim: int, latent_dim: int, output_dim: int):
        super().__init__()
        self.linear1 = nn.Linear(input_dim, latent_dim)
        self.act = nn.ReLU()
        self.linear2 = nn.Linear(latent_dim, output_dim)

    def forward(self, x):
        x = self.linear1(x)
        x = self.act(x)
        x = self.linear2(x)
        return x
    
    def set_optim(self, learning_rate: float, momentum: float):
        self.optim = optim.Adam(self.parameters(), lr=learning_rate, momentum=momentum)

    def get_loss_fn(self):
        return nn.MSELoss()
    
    def train_model(self, epochs: int, data_loader: data.DataLoader):
        self.train()
        loss_fn = self.get_loss_fn()

        for epoch in range(epochs):
            for inputs, labels in data_loader:
                outputs = self(inputs)
                loss = loss_fn(outputs, labels)
                loss.backward()
                self.optim.step()
                self.optim.zero_grad()

            if epoch % 10 == 1:
                print(f"Epoch: {epoch}, loss: {loss.item():.3}")
    
    def evaluate_model(self, data_loader: data.DataLoader):
        self.eval()
        true_predictions, predicitons_amount = 0., 0.

        with torch.no_grad():
            for inputs, labels in data_loader:
                predictions = self(inputs)
                predictions = predictions.squeeze(dim=1)
                true_predictions += (predictions == labels).sum()
                predicitons_amount += labels.shape[0]

            accuracy = 100.0 * true_predictions / predicitons_amount
        
        print(f"Accuracy of the model: {accuracy:4.2f}%")

In [ ]:
input_dim = 64
latent_dim = 256
output_dim = 8
learning_rate = 0.001
momentum = 0.5

In [ ]:
model = NeuralNet(input_dim=input_dim, latent_dim=latent_dim, output_dim=output_dim)
model.set_optim(learning_rate=learning_rate, momentum=momentum)
loss_fn = model.get_loss_fn()

## **Training**

In [ ]:
model.train_model(epochs=150, data_loader=[])